In [1]:
!pip install python-dotenv 

In [2]:
import os
from dotenv import load_dotenv
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.llms.openai import OpenAI

In [3]:
from langchain_ollama.llms import OllamaLLM

llm = OllamaLLM(model="llama3.2:latest")

Settings.llm = llm
from llama_index.embeddings.ollama import OllamaEmbedding
Settings.embed_model = OllamaEmbedding(model_name="nomic-embed-text")

In [4]:
# Cargar documentos
data_dir = "./data"
try:
    documents = SimpleDirectoryReader(data_dir).load_data()
    if not documents:
        print(f"No se encontraron documentos en {data_dir}")
        exit()
    print(f"Se cargaron {len(documents)} documento(s).")
except Exception as e:
    print(f"Error cargando documentos: {e}")
    exit()

Se cargaron 2 documento(s).


In [5]:
print("Creando el índice...")
try:
    index = VectorStoreIndex.from_documents(documents)
    print("Índice creado.")
except Exception as e:
    print(f"Error creando el índice: {e}")
    exit()

Creando el índice...
Índice creado.


In [9]:
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=1,  # Recuperar los 3 fragmentos más similares
)

print(retriever)

In [10]:
consulta_test = "¿Qué es Langchain?"
nodos_recuperados = retriever.retrieve(consulta_test)

print(f"\n--- Fase de Recuperación para: '{consulta_test}' ---")
if nodos_recuperados:
    print(f"Se recuperaron {len(nodos_recuperados)} nodos:")
    for i, nodo in enumerate(nodos_recuperados):
        print(f"\nNodo {i+1} (Score: {nodo.score:.4f}):")
        print(nodo.get_content()[:250] + "...") # Imprime los primeros 250 caracteres del nodo
else:
    print("No se recuperaron nodos.")
print("---------------------------------------------")


--- Fase de Recuperación para: '¿Qué es Langchain?' ---
Se recuperaron 1 nodos:

Nodo 1 (Score: 0.6868):
Langchain es otro framework popular para construir aplicaciones con LLMs.
Ofrece componentes modulares como cadenas, agentes y herramientas de memoria.
Se puede integrar con Llama Index para flujos de trabajo RAG más complejos.
Crew AI y LangGraph so...
---------------------------------------------


In [13]:
response_synthesizer = get_response_synthesizer(
    response_mode="refine" # Puedes experimentar con diferentes modos
)

query_engine_rag = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

In [14]:
pregunta_rag = "Explícame brevemente qué es Langchain y cómo se relaciona con Llama Index."
print(f"\n--- Fase de Generación Aumentada para: '{pregunta_rag}' ---")


--- Fase de Generación Aumentada para: 'Explícame brevemente qué es Langchain y cómo se relaciona con Llama Index.' ---


In [ ]:
try:
    respuesta_rag = query_engine_rag.query(pregunta_rag)
    print(f"Respuesta del sistema RAG:\n{respuesta_rag}")

    print("\nNodos fuente utilizados para la respuesta:")
    for nodo_fuente in respuesta_rag.source_nodes:
        print(f"Score: {nodo_fuente.score:.4f}, Contenido: {nodo_fuente.text[:100]}...")

except Exception as e:
    print(f"Error durante la consulta RAG: {e}")
print("-----------------------------------------------------")

Respuesta del sistema RAG:
Langchain es un framework que permite la construcción de aplicaciones utilizando LLMs (Modelos de Lenguaje Profundos) mediante componentes modulares como cadenas, agentes y herramientas de memoria. También se puede integrar con LangGraph para crear flujos de trabajo RAG más complejos.

En cuanto a Llama Index, no hay información directa sobre su relación con Langchain en el contexto proporcionado. Sin embargo, se menciona que Langchain se puede integrar con Llama Index para flujos de trabajo RAG más complejos, lo que sugiere que Langchain es compatible con Llama Index y ofrece una forma de expandir sus capacidades.

En resumen, Langchain es un framework que ofrece componentes modulares para la construcción de aplicaciones con LLMs, mientras que su relación con Llama Index se centra en la integración de flujos de trabajo RAG más complejos.

Nodos fuente utilizados para la respuesta:
Score: 0.7453, Contenido: Langchain es otro framework popular para construir a

In [17]:
llm_directo = Settings.llm # Usamos el LLM configurado globalmente
pregunta_directa_llm = "Explícame brevemente qué es Langchain y cómo se relaciona con Llama Index."

print(f"\n--- Consulta Directa al LLM (sin RAG) para: '{pregunta_directa_llm}' ---")
try:
    respuesta_directa_llm = llm_directo.complete(pregunta_directa_llm)
    print(f"Respuesta directa del LLM:\n{respuesta_directa_llm.text}")
except Exception as e:
    print(f"Error durante la consulta directa al LLM: {e}")
print("-------------------------------------------------------------")


--- Consulta Directa al LLM (sin RAG) para: 'Explícame brevemente qué es Langchain y cómo se relaciona con Llama Index.' ---
Respuesta directa del LLM:
¡Hola! Me alegra explicarte sobre Langchain y su relación con Llama Index.

**¿Qué es Langchain?**

Langchain es una plataforma de software que permite a los desarrolladores crear modelos de lenguaje más sofisticados utilizando técnicas de procesamiento de lenguaje natural (NLP). Langchain se enfoca en proporcionar herramientas y frameworks para entrenar y utilizar modelos de lenguaje como si fueran "paquetes" reutilizables, lo que facilita la creación de aplicaciones de NLP más complejas.

**¿Qué es Llama Index?**

Llama Index es un subproyecto de Langchain que se enfoca en el desarrollo de índices de modelos de lenguaje (LLM). Los índices de modelo son estructuras de datos que permiten a los modelos de lenguaje acceder rápidamente a la información relevante y relacionada con una entrada de texto.

**Relación entre Langchain y Llama I